### 2. Pretrain on OULU-CASIA

### (1)Create dataset

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
import torch

In [ ]:
from __future__ import division
# import dlib
import time
import sys
import torch.utils.data as data
import cv2
import os
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable  
from PIL import Image
from torch.nn import functional as F
from torch.utils.data import Dataset 
import numpy as np
from torch.utils import data as Data
from tqdm import tqdm
import time

import torchvision.models as models

from ops.basic_ops import ConsensusModule

In [ ]:
from torchvision import  transforms

In [ ]:
from tfennet import *
get_ipython().run_line_magic('matplotlib', 'notebook')

import visdom

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
vis =  visdom.Visdom(env=u'rss',use_incoming_socket=False)


In [ ]:
class Mydataset(data.Dataset):
    '''
    the format of sampleLib:[tensor,label],[tensor,label],[tensor,label].......
    
    '''
    def __init__(self,sampleLib):
        self.sampleLib=sampleLib
#         self.transform = transforms.Compose([
#                                                transforms.ToTensor()
#                                            ])
    def __getitem__(self,index):
        [sample,label] = self.sampleLib[index]
        return sample, label

    def __len__(self):
        return len(self.sampleLib)
    


In [ ]:
sampleLib = torch.load('/.../oulu_group_0.pkl') 

In [ ]:
transformed_dataset = Mydataset(sampleLib = torch.load('/.../oulu_trans_video_tensor_list.pkl') )
# check loading
# 查看数据的导入是否成功
assert(transformed_dataset[0][0].shape == torch.Size([7, 3, 224, 224]))
assert(transformed_dataset[0][1] == 0)
assert(len(transformed_dataset) == 480)

In [ ]:
from tqdm import tqdm
# split into 10 groups for cross-validation
# 分成10个group存起来
sampleLib = torch.load('/.../oulu_trans_video_tensor_list.pkl')
for i in tqdm(range(10)):
    sampleLib_group_i = sampleLib[48*i : 48*(i+1)]
    sampleLib_no_i = sampleLib[0:48*i]+ sampleLib[48*(i+1):480]
    dataset_group_i = Mydataset(sampleLib_group_i)
    dataset_no_i = Mydataset(sampleLib_no_i)
    torch.save(dataset_group_i, '/.../oulu_group_{}.pkl'.format(i))
    torch.save(dataset_no_i, '/.../oulu_group_no_{}.pkl'.format(i))

In [ ]:
i = 0
one_train = torch.load('/.../oulu_group_no_{}.pkl'.format(i))
one_test = torch.load('/.../oulu_group_{}.pkl'.format(i))
assert(one_train[0][0].shape == torch.Size([7, 3, 224, 224]))
assert(one_train[0][1] == 0)
assert(len(one_train) == 480-48)

### (2)Pretrain on OULU-CASIA 在Oulu数据集上pretrain表情模型TFEN

* input_train ='/.../oulu_group_no_{}.pkl'.format(i)
* input_test ='/.../oulu_group_{}.pkl'.format(i))

* input_all = Mydataset(sampleLib = torch.load('/.../oulu_trans_video_tensor_list.pkl') )

In [ ]:
def adjust_learning_rate(optimizer, epoch, lr):
    """Sets the learning rate to the initial LR decayed by 10 every SSTEP epochs"""
    lr = lr * (0.01 ** (epoch // sstep))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def validation(test_loader,net):
    pred_list = []
    test_y_list = []
    for i in range(len(test_dataset)):
        test_y_list.append(test_dataset[i][1])
    #     test_loader.size
    for step, (test_x,test_y) in enumerate(test_loader):
#         test_x = test_x.to(DEVICE)
        test_x = test_x.cuda()
        test_y =test_y.numpy()

        test_output = net(test_x)                   
        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()

        pred_list +=  list(pred_y  )

    pred_ay = np.array(pred_list)
    test_y_ay = np.array(test_y_list)

    accu = float((pred_ay ==test_y_ay).astype(int).sum())/test_y_ay.size
    return accu


In [ ]:
# 10-fold cross-validation
time_start=time.time()


loss_10 = []
accu_10 = []
net_10 = []
group_list = [0,1,2,3,4,5,6,7,8,9]
for i in group_list:
    BATCH_SIZE =16
    print('start loading data')
    train_dataset = torch.load('/.../oulu_group_no_{}.pkl'.format(i))
    
    train_loader = Data.DataLoader(
        dataset=train_dataset,      
        batch_size=BATCH_SIZE,      
        shuffle = False,               
    )
    test_dataset = torch.load('/.../oulu_group_{}.pkl'.format(i))
    test_loader = Data.DataLoader(
        dataset=test_dataset,     
        batch_size=BATCH_SIZE,     
        shuffle = False,               
    )
    print('finish loading data')
    net = TFEN_6()
#     net = torch.nn.DataParallel(net.cuda(), device_ids=[3])
    net = net.cuda()
    
    EPOCH = 400
    LR = 0.001
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    sstep = 70

    optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    loss_func = torch.nn.CrossEntropyLoss()
    loss_his = []
    accu_list = []
    
    
    for epoch in range(EPOCH):
        
#         print('====FOLDER:{}====EPOCH:{}/{}===='.format(i,epoch,EPOCH))
        
        adjust_learning_rate(optimizer, epoch, LR)
        
        for step, (batch_x,batch_y) in enumerate(train_loader):

            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()

            out = net(batch_x)
            loss = loss_func(out,batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_his.append(loss.data.cpu().numpy())
                        
            if step % 50 == 0:
                accuracy = validation(test_loader,net)
                accu_list.append(accuracy)
                print('Step:{}, Accuracy Rate:{}'.format(step,accuracy))
                
                
                
                if accuracy == max(accu_list):
                    net_best = net
                
                
                
                x= torch.Tensor([len(accu_list)])
                y=torch.Tensor([accuracy])
                vis.line(X=x, Y=y, 
                         win='oulu_{}'.format(i), 
                    opts=dict(
#                         legend=['oulu_adjustlr0.01_sstep70_2_{}'.format(i)],
                        title='oulu_{}'.format(i),
                        width=800,
                        height=800,
                             ),
                         update='append' if len(accu_list)>0 else None)

    loss_10.append(loss_his)
    accu_10.append(accu_list)
    net_10.append(net_best)
    


time_end=time.time()
m, s = divmod(time_end-time_start, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))

m, s = divmod(time_end-time_start, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))

torch.save(net_10,'torch14_adjustlr0.01_sstep7.pkl')
torch.save(accu_10,'torch14_adjustlr0.01_sstep70.pkl')
torch.save(loss_10,'torch14_adjustlr0.01_sstep70.pkl')
print(time.strftime('torch14_adjustlr0.01_sstep70.pkl ===> %Y-%m-%d %H:%M:%S', time.localtime()))

